In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [2]:
class LineChartCreator:
    def __init__(self, df, building_no, zone_name, date_range=None, agg='day'):
        self._df = df.copy()
        self._building_no = building_no
        self._date_range = date_range
        self._zone_name = zone_name
        
        self.line1_color = px.colors.qualitative.Plotly[1]
        self.line2_color = px.colors.qualitative.Plotly[2]
        self.marker_thick = 6
        self.marker_line_color = 'black'
        self.line_thick = 0.5
        self.opacity1 = 1
        self.mode = 'markers+lines'
        
        self.shading_color = px.colors.qualitative.Pastel[5]
        
        self._fig = self.update_fig(df, building_no, zone_name, date_range, agg)
        
        
    @property
    def df(self):
        return self._df
    
    @property
    def building_no(self):
        return self._building_no
    
    @property
    def date_range(self):
        return self._date_range
    
    @property
    def zone_name(self):
        return self._zone_name
    
    @property
    def fig(self):
        return self._fig
    
    def update_plot_styling(self, line1_color=None, line2_color=None, marker_line_color=None, marker_thick=None, line_thick=None, opacity1=None, mode=None, shading_color=None):
        if line1_color is not None:
            self.line1_color = line1_color
        if line2_color is not None:
            self.line2_color = line2_color
        if marker_line_color is not None:
            self.marker_line_color = marker_line_color
        if marker_thick is not None:
            self.marker_thick = marker_thick
        if line2_color is not None:
            self.line2_color = line2_color
        if line_thick is not None:
            self.line_thick = line_thick
        if opacity1 is not None:
            self.opacity1 = opacity1
        if mode is not None:
            self.mode = mode
        if shading_color is not None:
            self.shading_color = shading_color
    
    def update_fig(self, df, building_no, zone_name, date_range=None, agg='day'):
        # Suppress Plotly Deprecated dt warning
        import warnings
        warnings.simplefilter("ignore", category=FutureWarning)

        # Subset data
        df = df[(df.building_no==building_no)&(df.Fan_status=='On')&(df.Zone_name==zone_name)]
        
        # Aggregate data
        temp = pd.Series(pd.to_datetime(df.Datetime, errors='coerce').values, index=df.index, name='Datetime')
        # grouping checkbox values
        group_mapping = {
            'day': temp.dt.date,
            'week': temp.dt.to_period('W').apply(lambda r: r.start_time).dt.date,
            'month': temp.dt.to_period('M').apply(lambda r: r.start_time).dt.date
        }
        df = df.groupby(['building_no','Zone_name','Season','Faulty','Fan_status',group_mapping[agg]])[['Fan_time_diff','Predicted']].sum().reset_index().sort_values(by='Datetime')
         
        fig = go.Figure()
            
        # Add main plots
        fig.add_trace(
            go.Scattergl(
                x=df.Datetime,
                y=df.Fan_time_diff,
                marker=dict(
                    color=self.line1_color,
                    size=self.marker_thick,
                    line=dict(
                        color=self.marker_line_color,
                        width=self.line_thick
                    )
                ),
                mode=self.mode,
                name='Actual'
            )
        )
        fig.add_trace(
            go.Scattergl(
                x=df.Datetime,
                y=df.Predicted,
                marker=dict(
                    color=self.line2_color,
                    size=self.marker_thick,
                    line=dict(
                        color=self.marker_line_color,
                        width=self.line_thick
                    )
                ),
                mode=self.mode,
                name='Predicted'
            )
        )
        
        # Add background of Date Range selection
        if date_range is None:
            date_range = [df.Datetime.min(), df.Datetime.max()]
            
        fig.add_shape(
            type='rect',
            xref='x', yref='paper',
            x0=date_range[0], x1=date_range[1], y0=0, y1=1,
            fillcolor=self.shading_color, opacity=0.2, layer='below', line_width=0,
        )
            
        # Add axis titles
        agg_map = {'day':'Daily', 'week':'Weekly', 'month':'Monthly'}
        fig.update_layout(
            xaxis=dict(title=f'<b>{self.zone_name} All time Fan Usage</b>'), 
            yaxis=dict(title=f'<b>Total {agg_map[agg]} Fan On Time (mins)</b>'),
            hovermode='x unified'
            )
        
        return fig

In [3]:
df = pd.read_parquet(r'summary_all.parquet')

In [4]:
display(LineChartCreator(df, 1, 'Acu102').fig)

In [5]:
display(LineChartCreator(df,1,'Acu101').update_fig(df, 1, 'Acu102', agg='week'))

In [241]:
import datetime
display(LineChartCreator(df,1,'Acu101').update_fig(df, 1, 'Acu101', [datetime.date(2023, 7, 4), datetime.date(2023, 8, 1)], 'month'))